In [ ]:
#Provisional Machine Learning Model by Claire Golden
#Goal) input: pollution and air quality index values, 
#Goal) output: predicted values of air index quality based on existing patterns of pollutants
#Model) Linear Regression: estimating that pollutants steadily or drastically increase with time, also estimating that these 4 main pollutants directly effect air quality
#Training) will use data on 4 main pollutant levels from 2000-2016 and the corresponding air quality index values to train 

In [2]:
# Load libraries
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [10]:
pollution_df = pd.read_csv(r'resources/us_pollution_2000_2016.csv')
pollution_df.head()
                           

,usstate,county,city,local_date,no2avg,no2maxvalue,no2maxhour,no2aqi,o3avg,o3maxvalue,o3maxhour,o3aqi,so2avg,so2maxvalue,so2maxhour,so2aqi,coavg,comaxvalue,comaxhour,coaqi
0,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,3.000000,9.0,21,13.0,1.145833,4.2,21,NaN
1,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,3.000000,9.0,21,13.0,0.878947,2.2,23,25.0
2,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,2.975000,6.6,23,NaN,1.145833,4.2,21,NaN
3,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,2.975000,6.6,23,NaN,0.878947,2.2,23,25.0
4,Arizona,Maricopa,Phoenix,2000-01-02,22.958333,36.0,19,34,0.013375,0.032,10,27,1.958333,3.0,22,4.0,0.850000,1.6,23,NaN


In [11]:
pollution_df.dtypes

usstate         object
county          object
city            object
local_date      object
no2avg         float64
no2maxvalue    float64
no2maxhour       int64
no2aqi           int64
o3avg          float64
o3maxvalue     float64
o3maxhour        int64
o3aqi            int64
so2avg         float64
so2maxvalue    float64
so2maxhour       int64
so2aqi         float64
coavg          float64
comaxvalue     float64
comaxhour        int64
coaqi          float64
dtype: object

In [12]:
print(pollution_df.shape)

(1746661, 20)


In [13]:
pollution_df.describe()

,no2avg,no2maxvalue,no2maxhour,no2aqi,o3avg,o3maxvalue,o3maxhour,o3aqi,so2avg,so2maxvalue,so2maxhour,so2aqi,coavg,comaxvalue,comaxhour,coaqi
count,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,873754.000000,1.746661e+06,1.746661e+06,1.746661e+06,873338.000000
mean,1.282193e+01,2.541485e+01,1.173102e+01,2.389822e+01,2.612485e-02,3.920331e-02,1.017053e+01,3.605012e+01,1.870364e+00,4.492185e+00,9.664906e+00,7.115945,3.682177e-01,6.201067e-01,7.875026e+00,5.996595
std,9.504814e+00,1.599963e+01,7.877501e+00,1.516280e+01,1.136974e-02,1.534362e-02,4.003144e+00,1.978042e+01,2.760435e+00,7.679866e+00,6.731228e+00,11.937473,3.140231e-01,6.439361e-01,7.978844e+00,5.851836
min,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000,-4.375000e-01,-4.000000e-01,0.000000e+00,0.000000
25%,5.750000e+00,1.300000e+01,5.000000e+00,1.200000e+01,1.787500e-02,2.900000e-02,9.000000e+00,2.500000e+01,2.565220e-01,8.000000e-01,5.000000e+00,1.000000,1.834580e-01,2.920000e-01,0.000000e+00,2.000000
50%,1.073913e+01,2.400000e+01,9.000000e+00,2.300000e+01,2.587500e-02,3.800000e-02,1.000000e+01,3.300000e+01,9.875000e-01,2.000000e+00,8.000000e+00,3.000000,2.926250e-01,4.000000e-01,6.000000e+00,5.000000
75%,1.771364e+01,3.570000e+01,2.000000e+01,3.300000e+01,3.391700e-02,4.800000e-02,1.100000e+01,4.200000e+01,2.325000e+00,5.000000e+00,1.400000e+01,9.000000,4.666670e-01,8.000000e-01,1.300000e+01,8.000000
max,1.395417e+02,2.670000e+02,2.300000e+01,1.320000e+02,9.508300e-02,1.410000e-01,2.300000e+01,2.180000e+02,3.216250e+02,3.510000e+02,2.300000e+01,200.000000,7.508333e+00,1.990000e+01,2.300000e+01,201.000000


In [14]:
list(pollution_df.columns)

['usstate',
 'county',
 'city',
 'local_date',
 'no2avg',
 'no2maxvalue',
 'no2maxhour',
 'no2aqi',
 'o3avg',
 'o3maxvalue',
 'o3maxhour',
 'o3aqi',
 'so2avg',
 'so2maxvalue',
 'so2maxhour',
 'so2aqi',
 'coavg',
 'comaxvalue',
 'comaxhour',
 'coaqi']

In [ ]:
#Insert Data Cleaning Code here 

In [ ]:
#Separate clean dataframe into features and target variable 
X = cleandf.iloc[].values
y = cleandf.iloc[].values

In [ ]:
#Split into training and testing sets
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=1)

In [ ]:
#Linear regression model 
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
#Visualizing Linear Regression model, training set
lin_viz_train = plt
lin_viz_train.scatter(X_train, y_train, color='blue')
lin_viz_train.plot(X_train, regressor.predict(X_train), color='green')
lin_viz_train.title('')
lin_viz_train.xlabel('')
lin_viz_train.ylabel('')
lin_viz_train.show()

In [ ]:
#Visualizing Linear Regression model, testing set
lin_viz_test = plt
lin_viz_test.scatter(X_test, y_test, color='blue')
lin_viz_test.plot(X_train, regressor.predict(X_train), color='green')
lin_viz_test.title('')
lin_viz_test.xlabel('')
lin_viz_test.ylabel('')
lin_viz_test.show()

In [ ]:
#Analyze Linear Regression model
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
#Logistic Reegression Model
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

In [ ]:
#Visualizing Logistic Regression model, training set
log_viz_train = plt
log_viz_train.scatter(X_train, y_train, color='blue')
log_viz_train.plot(X_train, regressor.predict(X_train), color='green')
log_viz_train.title('')
log_viz_train.xlabel('')
log_viz_train.ylabel('')
log_viz_train.show()

In [ ]:
#Visualizing Logistic Regression model, testing set
log_viz_test = plt
log_viz_test.scatter(X_test, y_test, color='blue')
log_viz_test.plot(X_train, regressor.predict(X_train), color='green')
log_viz_test.title('')
log_viz_test.xlabel('')
log_viz_test.ylabel('')
log_viz_test.show()

In [ ]:
#Analyze Logistic Regression Model
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))